# 4 Network

## 4.1 CNN构建及网络参数的使用
在Network类中，我们定义了两个卷积层和三个线性层；两个主要的部分封装在其中，即前向函数的定义和权重张量；每个层中权重张量包含了随着我们的网络在训练过程中学习而更新的权重值（这就是在网络类中将层定义为类属性的原因）；在Module类中，pytorch可以跟踪每一层的权重张量，由于我们在创建Network类时扩展了Module类，也就自动继承了该功能。
- Parameter和Argument的区别：
    - Parameter在函数定义中使用，可将其看作是占位符；(形参)
    - Argument是当函数被调用时传递给函数的实际值；（实参）
- Parameter的两种类型：
    - 1.Hyperparameters:其值是手动和任意确定的；要构建神经网络：kernel_size, out_channels, out_features都需要手动选择
    - 2.Data dependent Hyperparameters:其值是依赖于数据的参数
        - 该参数位于网络的开始或末端，即第一个卷积层的输入通道和最后一个卷积层的输出特征图
        - 第一个卷积层的输入通道依赖于构成训练集的图像内部的彩色通道的数量（灰度图像是1，彩色图像是3）
        - 输出层的输出特征依赖于训练集中类的数量（fashion-MNIST数据集中的类型为10，则输出层的out_features=10）
        - 通常情况下，一层的输入是上一层的输出（即：卷积层中所有输入通道和线性层中的输入特征都依赖于上一层的数据）
- 当张量从卷积层传入线性层时，张量必须是flatten的

|     Parameter     |     Description      |
|:----------------------|:-------------------------|
|kernel_size| 设置滤波器的大小；滤波器的数量就是输出通道数|
|out_channels| 设置滤波器的数量，即为输出通道数|
|out_features| 设置输出张量的大小|

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

print(torch.__version__)
print(torchvision.__version__)

1.6.0
0.7.0


In [2]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # implement the forward pass
        return t

In [3]:
network = Network()
network

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

## 4.2 CNN的权重
* 可学习参数：是在训练过程中学习的参数，初值是选择的任意值，其值在网络学习的过程中以迭代的方式进行更新
* 说网络在学习是指：网络在学习参数的适合的值，适合的值就是能使损失函数最小化的值
* 可学习的参数是网络的权重，存在于每一层中
* 当我们扩展类的时候，我们会得到它的所有功能，为了得到它，我们可以添加额外的功能，也可覆盖现有的功能
* 在python中，所有特殊的面向对象的方法通常都有前双下划线和后双下划线（__init__, __repr__）

### 4.2.1 访问某一层的权重

In [4]:
network.conv1

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))

In [5]:
network.conv1.weight

Parameter containing:
tensor([[[[ 0.1797,  0.0765, -0.0372, -0.1412, -0.0842],
          [ 0.1012, -0.1364, -0.0281,  0.1101,  0.1991],
          [ 0.1393,  0.0511,  0.0083,  0.0407,  0.1463],
          [ 0.0428, -0.0529,  0.1675,  0.1147,  0.0563],
          [ 0.1560, -0.1279,  0.1985, -0.0862, -0.0925]]],


        [[[-0.1180, -0.1388,  0.1310, -0.0570,  0.1920],
          [ 0.0006,  0.0281, -0.1017,  0.1259, -0.1193],
          [ 0.0089,  0.0258,  0.1690, -0.0420, -0.1929],
          [-0.0572, -0.0012,  0.0401,  0.1847, -0.1560],
          [ 0.1317,  0.1314,  0.1013,  0.1963, -0.0810]]],


        [[[ 0.0682,  0.0445, -0.1794,  0.0146,  0.0390],
          [ 0.0921,  0.1916, -0.0424,  0.0943,  0.1528],
          [-0.1276, -0.0966, -0.0605,  0.0812, -0.1633],
          [-0.1664,  0.1237, -0.1140, -0.0153, -0.0667],
          [ 0.1088, -0.1658,  0.0326,  0.0254,  0.0872]]],


        [[[ 0.1244, -0.0275,  0.1337, -0.0330, -0.0359],
          [-0.0609, -0.1065,  0.0085,  0.0161, -0.1593

In [6]:
network.conv1.weight.shape

torch.Size([6, 1, 5, 5])

- 6 表示有 6 个卷积核
- 1 表示卷积核的深度为 1
- 5，5 表示卷积核的大小

### 4.2.1 访问所有的权重

In [7]:
for param in network.parameters():
    print(param.shape)

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([12, 6, 5, 5])
torch.Size([12])
torch.Size([120, 192])
torch.Size([120])
torch.Size([60, 120])
torch.Size([60])
torch.Size([10, 60])
torch.Size([10])


In [8]:
for name, param in network.named_parameters():
    print(name, '\t\t', param.shape)

conv1.weight 		 torch.Size([6, 1, 5, 5])
conv1.bias 		 torch.Size([6])
conv2.weight 		 torch.Size([12, 6, 5, 5])
conv2.bias 		 torch.Size([12])
fc1.weight 		 torch.Size([120, 192])
fc1.bias 		 torch.Size([120])
fc2.weight 		 torch.Size([60, 120])
fc2.bias 		 torch.Size([60])
out.weight 		 torch.Size([10, 60])
out.bias 		 torch.Size([10])


## 4.3 CNN前向方法的实现
* 前向方法的实现将使用我们在构造函数中定义的所有层
* 前向方法实际上是输入张量到预测的输出张量的映射

### 4.3.1 Input Layer
* 输入层是由输入数据决定的
* 输入层可以看做是恒等变换 f(x)=x
* 输入层通常是隐式存在的

In [9]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) conv layer1
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (3) conv layer2
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) linear layer1
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        
        # (5) linear layer2
        t = F.relu(self.fc2(t))
        
        # (6) output layer
        t = self.out(t)
        
        return t

## 4.4 单张图像的预测

### 4.4.1 前向传播(forward propagation)
* 是将输入张量转换为输出张量的过程（即：神经网络是将输入张量映射到输出张量的函数）
* 前向传播只是将输入张量传递给网络并从网络接收输出的过程的一个特殊名称

### 4.4.2 反向传播(back propagation)
* 反向传播通常在前向传播后发生
* 使用torch.set_grad_enabled(False)来关闭pytorch的梯度计算，这将阻止pytorch在我们的张量通过网络时构建一个计算图
* 计算图通过跟踪张量在网络中传播的每一个计算，来跟踪网络的映射；然后在训练过程中使用这个图来计算导数，也就是损失函数的梯度；关闭并非强制的，但可以减少内存。

In [10]:
train_set = torchvision.datasets.FashionMNIST(
    train=True,
    root='./data/FashionMNIST/',
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [11]:
sample = next(iter(train_set))
image, label = sample

In [12]:
image.shape

torch.Size([1, 28, 28])

In [13]:
network = Network()
pred = network(image.unsqueeze(0)) # [1, 28, 28] -> [1, 1, 28, 28]
print(pred.shape)
print(pred.argmax(dim=1))
print(label)

torch.Size([1, 10])
tensor([0])
9


In [14]:
# 要想将预测值用概率表示，可以使用softmax
print(F.softmax(pred, dim=1))

tensor([[0.1208, 0.0923, 0.1070, 0.0863, 0.0949, 0.1044, 0.1062, 0.0981, 0.1002, 0.0898]], grad_fn=<SoftmaxBackward>)


## 4.5 单批次图像预测

In [15]:
torch.set_printoptions(linewidth=120)

In [16]:
data_loader = torch.utils.data.DataLoader(train_set, batch_size=10)
batch = next(iter(data_loader))
images, labels = batch

In [17]:
preds = network(images)
preds

tensor([[ 0.1635, -0.1054,  0.0422, -0.1723, -0.0774,  0.0174,  0.0348, -0.0442, -0.0236, -0.1331],
        [ 0.1547, -0.0904,  0.0449, -0.1830, -0.0714,  0.0238,  0.0267, -0.0584, -0.0172, -0.1337],
        [ 0.1529, -0.0983,  0.0406, -0.1599, -0.0885,  0.0286,  0.0153, -0.0557, -0.0162, -0.1244],
        [ 0.1519, -0.0950,  0.0414, -0.1699, -0.0811,  0.0311,  0.0193, -0.0571, -0.0156, -0.1310],
        [ 0.1533, -0.1012,  0.0464, -0.1820, -0.0754,  0.0167,  0.0245, -0.0599, -0.0129, -0.1313],
        [ 0.1530, -0.0907,  0.0439, -0.1781, -0.0725,  0.0306,  0.0268, -0.0584, -0.0162, -0.1359],
        [ 0.1650, -0.0962,  0.0376, -0.1593, -0.0800,  0.0263,  0.0323, -0.0513, -0.0251, -0.1378],
        [ 0.1549, -0.0937,  0.0462, -0.1802, -0.0670,  0.0263,  0.0306, -0.0573, -0.0166, -0.1401],
        [ 0.1575, -0.0986,  0.0425, -0.1618, -0.0804,  0.0309,  0.0182, -0.0577, -0.0168, -0.1329],
        [ 0.1652, -0.0970,  0.0387, -0.1628, -0.0796,  0.0302,  0.0274, -0.0442, -0.0173, -0.1332]],

In [18]:
print(preds.argmax(dim=1))
print(labels)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5])


In [19]:
print(F.softmax(preds, dim=1))

tensor([[0.1208, 0.0923, 0.1070, 0.0863, 0.0949, 0.1044, 0.1062, 0.0981, 0.1002, 0.0898],
        [0.1198, 0.0938, 0.1074, 0.0855, 0.0956, 0.1051, 0.1054, 0.0968, 0.1009, 0.0898],
        [0.1197, 0.0931, 0.1069, 0.0875, 0.0940, 0.1057, 0.1043, 0.0971, 0.1010, 0.0907],
        [0.1195, 0.0934, 0.1070, 0.0866, 0.0947, 0.1059, 0.1047, 0.0970, 0.1011, 0.0901],
        [0.1199, 0.0929, 0.1077, 0.0857, 0.0954, 0.1046, 0.1054, 0.0968, 0.1015, 0.0902],
        [0.1195, 0.0937, 0.1072, 0.0858, 0.0954, 0.1058, 0.1054, 0.0968, 0.1009, 0.0895],
        [0.1209, 0.0931, 0.1064, 0.0874, 0.0946, 0.1052, 0.1059, 0.0974, 0.0999, 0.0893],
        [0.1197, 0.0934, 0.1074, 0.0856, 0.0959, 0.1053, 0.1057, 0.0968, 0.1009, 0.0892],
        [0.1201, 0.0930, 0.1071, 0.0873, 0.0947, 0.1058, 0.1045, 0.0969, 0.1009, 0.0898],
        [0.1207, 0.0929, 0.1064, 0.0869, 0.0945, 0.1055, 0.1052, 0.0979, 0.1006, 0.0896]], grad_fn=<SoftmaxBackward>)


## 4.6 输入张量在通过CNN的过程中的变化

### 4.6.1 CNN 输出特征图尺寸(正方形)
* 假设输入特征的大小为n x n
* 假设滤波器的大小为 f x f
* 令padding为p，步长stride为s
* 则输出特征图的大小为 $O = \frac{n - f + 2p}/{s} + 1$

### 4.6.2 CNN 输出特征图尺寸(非正方形)
* 假设输入特征的大小为 nh x nw
* 假设滤波器的大小为 fh x fw
* 令padding为p，步长stride为s
* 则输出特征图的高度为 $O_h = \frac{n_h - f + 2p}/{s} + 1$
* 输出特征图的宽度为 $O_w = \frac{n_w - f + 2p}/{s} + 1$